- Ggf. zu float16 konvertieren checken, ob finaler df mit time slices dann deutlich kleiner und performance testen
- Cross validation?
- Ensemble learning?

In [1]:
#Import data handling libraries
import pandas as pd
import numpy as np

In [2]:
#Specify directories

###local###
#get parent folder of current directory
parent_dir = '/Users/mf/Desktop/CS/Studies/7_Final_Project/Kaggle_M5PointPrediction'

#Directory resources
res_dir = parent_dir + '/res/'
src_dir = parent_dir + '/src/'
prc_dir = src_dir + 'processed_data/' # Processed data directory with pickled dataframes

###On Kaggle###
#res_dir = '/kaggle/input/m5-forecasting-accuracy/'
# ?? src_dir = '/kaggle/input/m5-forecasting-accuracy/src/'


print('Parent directory of current notebook: ' + parent_dir)
print('Resource directory: ' + res_dir)

Parent directory of current notebook: /Users/mf/Desktop/CS/Studies/7_Final_Project/Kaggle_M5PointPrediction
Resource directory: /Users/mf/Desktop/CS/Studies/7_Final_Project/Kaggle_M5PointPrediction/res/


In [3]:
# Create variables
BASE     = prc_dir +'df_1.pkl'
CALENDAR = prc_dir +'df_2.pkl'

In [4]:
# Read in df_train_conv from pickle file
df_all_data = pd.concat([pd.read_pickle(BASE),
           pd.read_pickle(CALENDAR)], 
           axis=1)

In [5]:
# Splitting the data
TRAIN_END = 1913 - 28 # Train only until the 28 days before the end of the data
VAL_END = 1913

df_train = df_all_data[df_all_data['d'] <= TRAIN_END]
df_val = df_all_data[(df_all_data['d'] > TRAIN_END) & (df_all_data['d'] <= VAL_END)]
df_test = df_all_data[df_all_data['d'] > VAL_END]

In [6]:
from sklearn.preprocessing import MinMaxScaler
from keras.models import Model
from keras.layers import Input, LSTM, Dense
from keras.optimizers import Adam

In [7]:
# Define categorical and numerical columns
categorical_cols = ['item_id', 'dept_id', 'cat_id', 'store_id', 'state_id', 'wday', 
                    'event_name_1', 'event_type_1', 'event_name_2', 'event_type_2', 
                    'snap_CA', 'snap_TX', 'snap_WI', 'mday', 'week', 'month', 'year']
numerical_cols = ['sell_price']

target_col = 'sales_amount'

# Convert categorical columns to category dtype and encode with cat.codes
for col in categorical_cols:
    df_train[col] = df_train[col].astype('category').cat.codes

# Normalize numerical columns
scaler = MinMaxScaler()
df_train[numerical_cols] = scaler.fit_transform(df_train[numerical_cols].astype(np.float32))

# LSTM Data Preparation
time_steps = 7  # Number of time steps (can be tuned)
batch_size = 32  # Batch size for the generator


/var/folders/w1/_96k6h412s5d15f14hdtbbyc0000gn/T/ipykernel_48660/4075745869.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train[col] = df_train[col].astype('category').cat.codes
/var/folders/w1/_96k6h412s5d15f14hdtbbyc0000gn/T/ipykernel_48660/4075745869.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train[col] = df_train[col].astype('category').cat.codes
/var/folders/w1/_96k6h412s5d15f14hdtbbyc0000gn/T/ipykernel_48660/4075745869.py:11: SettingWithCopyWarning: 
A value is trying to be set 

In [8]:
# Custom RMSE loss function
def rmse(y_true, y_pred):
    return K.sqrt(K.mean(K.square(y_pred - y_true)))

In [ ]:
def create_sequences(df, target, time_steps=1):
    X, y = [], []
    for i in range(len(df) - time_steps):
        X.append(df.iloc[i:(i + time_steps), :-1].values)
        y.append(df.iloc[i + time_steps][target])
    return np.array(X), np.array(y)

# Create sequences
X_train, y_train = create_sequences(df_train, target_col, time_steps=time_steps)